In [1]:
# !pip install duckdb
import pandas as pd
import numpy as np
import duckdb

In [2]:
# !pip install langchain_openai
# !pip install openai
from openai import AzureOpenAI
from langchain_openai import AzureChatOpenAI
from langchain.tools.render import render_text_description_and_args
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate

In [3]:
import json
import re
from tqdm import tqdm

# Import data/ clean data

In [4]:
df = pd.read_csv('dataset_facebook-groups-scraper_2025-04-11_03-37-09-271.csv',encoding='utf-8')

C:\Users\user\AppData\Local\Temp\ipykernel_27868\550170161.py:1: DtypeWarning: Columns (13,14,42,43,45,48,50,51,52,53,54,59,61,62,66,68,69,70,71,72,77,79,80,84,86,87,88,89,90,95,97,98,102,104,105,106,107,108,113,115,116,120,122,123,124,125,126,131,133,134,137,139,140,141,142,143,148,150,151,155,157,158,159,160,161,166,168,169,173,175,176,177,178,179,184,186,187,191,193,194,195,196,197,202,204,205,209,212,213,216,217,221,222,226,227,231,232,241,242,243,274,277,293,304,310,311,312,313,314,315,316,317,318,319,320,321,323,324,325,326,327,330,335,337,338,339,340,341,346,348,349,353,355,356,357,358,359,364,366,367,371,373,374,375,376,377,383,384,388,393,394,395,396,397,402,404,405,409,411,412,413,414,415,420,422,423,427,433,434,436,437,449,477,478,479,481,482,483,488,489,490,491,495,496,497,498,499,500,501,511,512,513,514,515,516,517,518,519,520,521,522,523,526,529,530,531,532,534,536,539,543,545,546,547,549,550,553,554,555,557,560,564,565,566,567,568,569,571,572,573,588,589,590,592,594,595,

In [5]:
df.head()

,attachments/0/__isMedia,attachments/0/__typename,attachments/0/accent_color,attachments/0/animated_image_caption,attachments/0/audio_availability,attachments/0/audio_settings,attachments/0/audio_user_preferred_language,attachments/0/autoplay_gating_result,attachments/0/breakingStatus,attachments/0/broadcast_id,...,topComments/1/profileId,topComments/1/profileName,topComments/1/profilePicture,topComments/1/profileUrl,topComments/1/text,topComments/1/threadingDepth,topReactionsCount,url,user/id,user/name
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,https://www.facebook.com/groups/30194376809621...,pfbid02yTDy4LVpH6CW1N6ZnMnD87AP4eDiXmA6WsdEdwa...,พัชรินทร์ ออรคิดส์
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,https://www.facebook.com/groups/30194376809621...,100005467236869,Anna Tongdee
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,https://www.facebook.com/groups/30194376809621...,pfbid023aWEFc2BmSXymPks5ytWHgwan2JpfbS5t1zwKAP...,Chanborom Garden
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,https://www.facebook.com/groups/30194376809621...,pfbid02PqPQTuTdArFE3TLPmF2XRPABUAWmpCDvD3SbWEG...,วิเชียร ห้วยไชย
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,https://www.facebook.com/groups/30194376809621...,100048556321990,Nuy Youme


In [6]:
df_cleaned = duckdb.query("""
select 
grouptitle as "group"
, cast(time as date) as date
, id as post_id
, title
, coalesce(likescount, 0) as reaction_count
from df
where grouptitle is not null
and grouptitle is not null
and time is not null
and facebookid is not null
and title is not null
and length(title) > 5
and length(title) <= 1000
and title not like 'this content%'
and title != ''
""").to_df()

In [7]:
# df_cleaned = duckdb.query("""
# select * from df_cleaned limit 100
# """).to_df()

In [8]:
df_cleaned.head()

,group,date,post_id,title,reaction_count
0,กลุ่มคนรักกล้วยไม้ป่า จะซื้อ จะขาย แจก แลกเปลี...,2025-04-11,UzpfSTEwMDA0ODU1NjMyMTk5MDpWSzoyNjA4NjUxMDYyNj...,003235/Bp1760/FR008/\n#เอื้องตาเหิน ไม้ป่าหายา...,0
1,กลุ่มคนรักกล้วยไม้ป่า จะซื้อ จะขาย แจก แลกเปลี...,2025-04-10,UzpfSTEwMDAwODExMDI5ODM4NDpWSzoyNjA4NjMzMDk5Mz...,(SOLD) เอื้องกุหลาบแดง,1
2,กลุ่มคนรักกล้วยไม้ป่า จะซื้อ จะขาย แจก แลกเปลี...,2025-04-10,UzpfSTEwMDA0ODU1NjMyMTk5MDpWSzoyNjA4MzAwMDYyNj...,003235/Bp1760/FR008\n🚩🚩ชุดรวม 100 บาท🚩🚩\nเอื้อ...,0
3,กลุ่มคนรักกล้วยไม้ป่า จะซื้อ จะขาย แจก แลกเปลี...,2025-04-10,UzpfSTEwMDA0ODU1NjMyMTk5MDpWSzoyNjA4Mjg2MDk5Mz...,003235/FR008/BP1760/\n🦁 สิง โต กลีบ หลอด/สิง โ...,0
4,กลุ่มคนรักกล้วยไม้ป่า จะซื้อ จะขาย แจก แลกเปลี...,2025-04-10,UzpfSTEwMDA0ODU1NjMyMTk5MDpWSzoyNjA4MDg4OTIyNz...,003235/BP1760/FR008\n#เอื้องลิ้นกระบือ ไม้ป่าห...,0


In [9]:
df_cleaned.shape

(1563, 5)

In [ ]:
client = AzureOpenAI(
    azure_endpoint="",
    api_version="2024-12-01-preview",
    api_key="",
)

In [ ]:
OrchidTaggingSchema = {
    "name": "analyze_orchid_post",
    "description": "Analyze a post about orchids and extract cleaned text, post category, and listed orchid species.",
    "parameters": {
        "type": "object",
        "properties": {
            "clean_post": {
                "type": "string",
                "description": "Cleaned text content without emojis, hashtags, unrelated symbols or number code or any code that\
                    is not related with context."
            },
            "post_category": {
                "type": "string",
                "description": "One of: Orchid-selling, Orchid-buying, Orchid tools-selling, Orchid tools-buying, Orchid discussion , Not Orchid or orchid tools-related"
                "Some post will contain only name of species then you can categoeize as a Orchid-selling",
                "enum": ["Orchid-selling", "Orchid-buying", "Orchid tools-selling",
                         "Orchid tools-buying", "Orchid discussion","Not Orchid or orchid tools-related"]
            },
            "orchid_species": {
                "type": "array",
                "items": {
                    "type": "string"
                },
                "description": "List of all Thai orchid species mentioned, for example ['สีดา','ไอยเรศ']."
            }
        },
        "required": ["clean_post", "post_category", "orchid_species"]
    }
}

In [12]:
def analyze_orchid_post(client, model_name, system_prompt, user_input, post_id):
    try:
        response = client.chat.completions.create(
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ],
            tools=[
                {
                    "type": "function",
                    "function": OrchidTaggingSchema
                }
            ],
            tool_choice="required",
            max_tokens=800,
            temperature=0.0,
            model=model_name
        )

        content = response.choices[0].message.tool_calls[0].function.arguments
        result = json.loads(content)
        result["post_id"] = post_id
        return result

    except Exception as e:
        return None

In [13]:
test_input = "กล้วยไม้ผมสวยไหมครับ มีทั้ง แวนด้า แคทลียา หวาย /d/few fe//ewfwe/f/"
test_id = '12412r512312'

In [14]:
analyze_orchid_post(client=client
                    , model_name='gpt-4.1-mini'
                    , system_prompt='You are a orchid expert assistant.'
                    , user_input=test_input, post_id=test_id)

{'clean_post': 'กล้วยไม้ผมสวยไหมครับ มีทั้ง แวนด้า แคทลียา หวาย',
 'post_category': 'Orchid discussion',
 'orchid_species': ['แวนด้า', 'แคทลียา', 'หวาย'],
 'post_id': '12412r512312'}

# Tag all post

In [15]:
results = []

In [16]:
for idx, row in tqdm(df_cleaned.iterrows(), total=len(df_cleaned), desc="Analyzing Orchid Posts"):
    title = row['title']
    post_id = row['post_id']
    
    result = analyze_orchid_post(client=client
                                 , model_name='gpt-4o-mini'
                                 , system_prompt='You are a orchid expert assistant.'
                                 , user_input=title
                                 , post_id=post_id)   
    if result:
        results.append(result)

Analyzing Orchid Posts: 100%|██████████| 1563/1563 [33:45<00:00,  1.30s/it] 


In [17]:
df_tagged_full = pd.DataFrame(results)
df_tagged_1 = df_tagged_full.drop(columns=['orchid_species'])
df_tagged_2 = df_tagged_full[['post_id', 'orchid_species']].explode('orchid_species').dropna()

# Final table

In [18]:
df_tagging_final = duckdb.query("""
select "group", cast("date" as date) as datetime, a.post_id, clean_post as title_clearned, post_category, reaction_count
from df_cleaned a
inner join df_tagged_1 b
on a.post_id = b.post_id
where post_category in ('Orchid-selling')
""").to_df()

In [19]:
df_species_final = duckdb.query("""
select b.*
from df_tagged_1 a
inner join df_tagged_2 b
on a.post_id = b.post_id
where post_category in ('Orchid-selling')
""").to_df()

# Export

In [21]:
df_tagging_final.to_excel('tagging.xlsx',index=False)

In [22]:
df_species_final.to_excel('species.xlsx',index=False)